<a href="https://colab.research.google.com/github/kadefue/MoEST/blob/main/MoEST_Refactored_Secondary_School_Enrollment_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Refactored MoEST Modeling for Secondary School Enrollment
"""

import os
import re
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import (
    RandomForestRegressor,
    HistGradientBoostingRegressor,
    GradientBoostingRegressor
)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from google.colab import drive

# =============================================================================
# CLASS 1: Data Loader & Cleaner
# =============================================================================

class MOESTDataLoader:
    """
    Handles mounting drive, loading CSVs, cleaning headers, handling types,
    and managing initial data quality checks.
    """
    def __init__(self, base_directory, exclude_keywords=None):
        self.base_directory = base_directory
        self.exclude_keywords = exclude_keywords if exclude_keywords else []
        self.dataframes = {}

    def mount_drive(self):
        drive.mount('/content/drive/')

    def get_file_list(self):
        all_files = [f for f in os.listdir(self.base_directory) if f.endswith('.csv')]
        filtered_files = []
        for file_name in all_files:
            if not any(keyword.lower() in file_name.lower() for keyword in self.exclude_keywords):
                filtered_files.append(file_name)
        return filtered_files

    def load_data(self):
        files = self.get_file_list()
        print(f"Found {len(files)} files to load.")
        for file_name in files:
            file_path = os.path.join(self.base_directory, file_name)
            df_name = file_name.replace('.csv', '')
            try:
                df = pd.read_csv(file_path)
                self.dataframes[df_name] = self._initial_clean(df)
                print(f"Loaded: {df_name}")
            except Exception as e:
                print(f"Error loading {file_name}: {e}")

    def _initial_clean(self, df):
        """Standardizes headers and cleans numeric columns."""
        df.columns = [str(col).strip().upper() for col in df.columns]

        for col in df.select_dtypes(include='object').columns:
            if df[col].astype(str).str.contains(',').any():
                cleaned = df[col].astype(str).str.replace(',', '', regex=False)
                converted = pd.to_numeric(cleaned, errors='coerce')
                if converted.notna().sum() > 0:
                    df[col] = converted

        unnamed = [c for c in df.columns if 'UNNAMED' in c]
        to_drop = [c for c in unnamed if df[c].isnull().mean() > 0.9]
        df.drop(columns=to_drop, inplace=True)
        return df

    def get_dataframe(self, name):
        return self.dataframes.get(name)

    def get_all_dataframes(self):
        return self.dataframes

# =============================================================================
# CLASS 2: Geography & Location Manager
# =============================================================================

class LocationManager:
    """
    Handles Geocoding, LGA Status merging, and Clustering.
    """
    def __init__(self, dataframes, geodata_path):
        self.dataframes = dataframes
        self.geodata_path = geodata_path
        self.geo_data = None
        self.lga_status_df = None

    def standardize_location_columns(self):
        for name, df in self.dataframes.items():
            reg_col = next((c for c in df.columns if c in ['REGION', 'REGON']), None)
            cou_col = next((c for c in df.columns if c in ['COUNCIL', 'DISTRICT', 'LGA NAME']), None)

            if reg_col and cou_col:
                df.rename(columns={reg_col: 'REGION', cou_col: 'COUNCIL'}, inplace=True)
                df['REGION'] = df['REGION'].astype(str).str.upper()
                df['COUNCIL'] = df['COUNCIL'].astype(str).str.upper()

    def merge_lga_status(self, lga_df_name='LGAs Urban and Rural Status'):
        if lga_df_name not in self.dataframes:
            print("LGA Status DataFrame not found.")
            return

        self.lga_status_df = self.dataframes[lga_df_name].copy()
        if 'REMARKS' in self.lga_status_df.columns:
            self.lga_status_df.drop(columns=['REMARKS'], inplace=True)
        self.lga_status_df.rename(columns={'CLASSIFICATION': 'LGA_STATUS'}, inplace=True)

        for name, df in self.dataframes.items():
            if name == lga_df_name: continue
            if 'REGION' in df.columns and 'COUNCIL' in df.columns:
                merged = pd.merge(df, self.lga_status_df, on=['REGION', 'COUNCIL'], how='left')
                self.dataframes[name] = merged
                print(f"Merged LGA Status into {name}")
        del self.dataframes[lga_df_name]

    def process_geocoding(self):
        if os.path.exists(self.geodata_path):
            print("Loading existing geodata...")
            self.geo_data = pd.read_csv(self.geodata_path)
        else:
            print("Generating new geodata...")
            self._fetch_geodata()

        self._apply_clustering()

        for name, df in self.dataframes.items():
            if 'REGION' in df.columns and 'COUNCIL' in df.columns:
                merged = pd.merge(df, self.geo_data[['REGION', 'COUNCIL', 'GEO_CLUSTER']],
                                  on=['REGION', 'COUNCIL'], how='left')
                self.dataframes[name] = merged
                print(f"Merged Geo Cluster into {name}")

    def _fetch_geodata(self):
        locs = []
        for df in self.dataframes.values():
            if 'REGION' in df.columns and 'COUNCIL' in df.columns:
                locs.append(df[['REGION', 'COUNCIL']])
        unique_locs = pd.concat(locs).drop_duplicates().reset_index(drop=True)

        geolocator = Nominatim(user_agent="moest_geo_mapper_v3")
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.0)

        lats, lons = [], []
        for idx, row in unique_locs.iterrows():
            query = f"{row['COUNCIL']}, {row['REGION']}, Tanzania"
            try:
                loc = geocode(query)
                if loc:
                    lats.append(loc.latitude)
                    lons.append(loc.longitude)
                else:
                    lats.append(None)
                    lons.append(None)
            except:
                lats.append(None)
                lons.append(None)

        unique_locs['LATITUDE'] = lats
        unique_locs['LONGITUDE'] = lons
        self.geo_data = unique_locs.dropna(subset=['LATITUDE', 'LONGITUDE'])
        self.geo_data.to_csv(self.geodata_path, index=False)

    def _apply_clustering(self):
        kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
        self.geo_data['GEO_CLUSTER'] = kmeans.fit_predict(self.geo_data[['LATITUDE', 'LONGITUDE']])

# =============================================================================
# CLASS 3: Feature Engineer
# =============================================================================

class FeatureEngineer:
    """
    Handles specific transformation logic for Secondary School Subjects.
    """
    @staticmethod
    def melt_subjects(df):
        id_vars = [c for c in ['YEAR', 'REGION', 'COUNCIL'] if c in df.columns]
        subject_cols = [c for c in df.columns if 'FORM ' in c and ' - ' in c]

        print(f"Melting {len(subject_cols)} subject columns...")
        long_df = df.melt(id_vars=id_vars, value_vars=subject_cols, var_name='RAW', value_name='ENROLLMENT')

        long_df['FORM_NUM'] = long_df['RAW'].str.extract(r'FORM (\d)').astype(int)
        long_df['SUBJECT'] = long_df['RAW'].str.split(' - ').str[1].str.strip()
        long_df.drop(columns=['RAW'], inplace=True)
        return long_df

    @staticmethod
    def merge_infrastructure(main_df, infrastructure_dfs):
        keys = ['YEAR', 'REGION', 'COUNCIL']
        df = main_df.copy()

        # Tables
        tables = infrastructure_dfs.get('tables')
        if tables is not None and 'AVAILABLE_TABLES' in tables.columns:
            df = df.merge(tables[keys + ['AVAILABLE_TABLES']], on=keys, how='left').fillna({'AVAILABLE_TABLES': 0})

        # Labs
        labs = infrastructure_dfs.get('labs')
        if labs is not None:
            lab_cols = [c for c in labs.columns if 'LABORATORY' in c]
            if lab_cols:
                labs['TOTAL_LABS'] = labs[lab_cols].sum(axis=1)
                df = df.merge(labs[keys + ['TOTAL_LABS']], on=keys, how='left').fillna({'TOTAL_LABS': 0})

        return df

    @staticmethod
    def create_lag_features(df):
        df = df.sort_values(['REGION', 'COUNCIL', 'SUBJECT', 'FORM_NUM', 'YEAR'])
        g = df.groupby(['REGION', 'COUNCIL', 'SUBJECT', 'FORM_NUM'])

        df['LAG_1'] = g['ENROLLMENT'].shift(1)
        df['LAG_2'] = g['ENROLLMENT'].shift(2)
        df['YOY_GROWTH'] = (df['ENROLLMENT'] - df['LAG_1']) / (df['LAG_1'] + 1e-5)
        df['IS_ELECTION_YEAR'] = df['YEAR'].isin([2015, 2020, 2025, 2030]).astype(int)

        return df.fillna(-1)

    @staticmethod
    def create_cohort_features(df):
        df['PREV_YEAR'] = df['YEAR'] - 1
        df['PREV_FORM'] = df['FORM_NUM'] - 1

        df['LOOKUP_KEY'] = (df['YEAR'].astype(str) + '_' + df['REGION'] + '_' +
                            df['COUNCIL'] + '_' + df['SUBJECT'] + '_' + df['FORM_NUM'].astype(str))

        lookup_dict = df.groupby('LOOKUP_KEY')['ENROLLMENT'].sum().to_dict()

        df['SEARCH_KEY'] = (df['PREV_YEAR'].astype(str) + '_' + df['REGION'] + '_' +
                            df['COUNCIL'] + '_' + df['SUBJECT'] + '_' + df['PREV_FORM'].astype(str))

        df['COHORT_LAG'] = df['SEARCH_KEY'].map(lookup_dict).fillna(-1)
        df.drop(columns=['PREV_YEAR', 'PREV_FORM', 'LOOKUP_KEY', 'SEARCH_KEY'], inplace=True)
        return df

# =============================================================================
# CLASS 4: Model Engine (Conditional Ensemble)
# =============================================================================

class EnrollmentModelEngine:
    """
    Manages Training, Conditional Selection, and Recursive Forecasting.
    Implements logic: Use Best Model ONLY unless others are within 60% performance.
    """
    def __init__(self, df):
        self.df = df
        self.models = {}
        self.encoders = {}
        self.features = [
            'YEAR', 'REGION_ENC', 'COUNCIL_ENC', 'SUBJECT_ENC', 'FORM_NUM',
            'AVAILABLE_TABLES', 'TOTAL_LABS', 'LAG_1', 'LAG_2',
            'YOY_GROWTH', 'COHORT_LAG', 'IS_ELECTION_YEAR'
        ]
        self.selected_model_keys = [] # Stores keys of models to be used for inference

    def preprocess(self):
        self.encoders['REG'] = LabelEncoder()
        self.encoders['COU'] = LabelEncoder()
        self.encoders['SUB'] = LabelEncoder()

        self.df['REGION_ENC'] = self.encoders['REG'].fit_transform(self.df['REGION'].astype(str))
        self.df['COUNCIL_ENC'] = self.encoders['COU'].fit_transform(self.df['COUNCIL'].astype(str))
        self.df['SUBJECT_ENC'] = self.encoders['SUB'].fit_transform(self.df['SUBJECT'].astype(str))

    def train_all_models(self, cutoff_year=2023):
        """Trains all 5 candidate models."""
        print(f"\nTraining all candidate models on Data <= {cutoff_year}...")
        train_df = self.df[self.df['YEAR'] <= cutoff_year]
        X = train_df[self.features]
        y = train_df['ENROLLMENT']

        # 1. XGBoost
        print("Training XGBoost...")
        self.models['XGB'] = xgb.XGBRegressor(n_estimators=300, max_depth=9, learning_rate=0.05, n_jobs=-1)
        self.models['XGB'].fit(X, y)

        # 2. LightGBM
        print("Training LightGBM...")
        self.models['LGB'] = lgb.LGBMRegressor(n_estimators=500, num_leaves=50, min_child_samples=10, learning_rate=0.1, verbose=-1)
        self.models['LGB'].fit(X, y)

        # 3. Random Forest
        print("Training Random Forest...")
        self.models['RF'] = RandomForestRegressor(n_estimators=300, max_depth=12, n_jobs=-1, random_state=42)
        self.models['RF'].fit(X, y)

        # 4. HistGradientBoosting (Scikit-Learn)
        print("Training HistGradientBoosting...")
        self.models['HGB'] = HistGradientBoostingRegressor(max_iter=500, learning_rate=0.1, max_depth=10)
        self.models['HGB'].fit(X, y)

        # 5. GradientBoosting (Standard)
        # Limiting estimators slightly for speed as GB is sequential and slow
        print("Training GradientBoosting...")
        self.models['GB'] = GradientBoostingRegressor(n_estimators=300, max_depth=9, learning_rate=0.05)
        self.models['GB'].fit(X, y)

        print("All models trained successfully.")
    def calculate_metrics(y_true, y_pred, model_name):
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)

        # Handle division by zero for MAPE
        mask = y_true != 0
        if mask.sum() > 0:
            mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
            accuracy = 100 - mape
        else:
            mape = np.nan
            accuracy = np.nan

        print(f"--- {model_name} Results ---")
        print(f"   > {model_name}: R2={r2:.4f}, MAE={mae:,.2f}, RMSE={rmse:,.2f}, Acc={accuracy:.2f}%")
        print("-" * 30)
        return rmse

    def evaluate_and_select_strategy(self, test_year_start=2024):
        """
        Evaluates models and applies the 60% rule:
        1. Find Best Model (Lowest RMSE).
        2. Identify models with RMSE <= Best RMSE * 1.6 (Within 60% of best).
        3. If only Best fits criteria -> Use Single Best.
        4. If others fit criteria -> Use Ensemble (Average of valid candidates).
        """
        test_df = self.df[self.df['YEAR'] >= test_year_start]
        if test_df.empty:
            print("No test data available. Defaulting to XGB only.")
            self.selected_model_keys = ['XGB']
            return

        X_test = test_df[self.features]
        y_true = test_df['ENROLLMENT']

        performance = {}
        print(f"\n--- Evaluation Results (Test Data {test_year_start}+) ---")

        # Evaluate all
        print("Evaluating all models...")
        for name, model in self.models.items():
            preds = model.predict(X_test)
            rmse = self.calculate_metrics(y_true, preds, name)
            performance[name] = rmse
            print(f"Model: {name} | RMSE: {rmse:,.2f}")

        # Find Best
        best_model_name = min(performance, key=performance.get)
        best_rmse = performance[best_model_name]
        print(f"\nBEST MODEL: {best_model_name} (RMSE: {best_rmse:,.2f})")

        # Apply Threshold Rule (RMSE within 40% of best)
        # "Performance at least 40% of best" usually means Error is not more than 40% higher.
        # Threshold = Best_RMSE + (0.40 * Best_RMSE) = 1.4 * Best_RMSE
        threshold = best_rmse * 1.4
        candidates = [name for name, score in performance.items() if score <= threshold]

        print(f"Selection Threshold (RMSE <= {threshold:,.2f})")
        print(f"Qualifying Models: {candidates}")

        # Logic: If others exist besides best -> Ensemble. Else -> Single.
        if len(candidates) > 1:
            self.selected_model_keys = candidates
            print(f"Strategy: ENSEMBLE (Average of {', '.join(candidates)})")
        else:
            self.selected_model_keys = [best_model_name]
            print(f"Strategy: SINGLE BEST MODEL ({best_model_name})")

    def _predict(self, X):
        """Predicts using the selected strategy."""
        preds = []
        for key in self.selected_model_keys:
            preds.append(self.models[key].predict(X))

        # Average the predictions of selected models
        return np.mean(preds, axis=0)

    def recursive_forecast(self, start_year, end_year):
        print(f"\nStarting Recursive Forecast ({start_year}-{end_year}) using {self.selected_model_keys}...")
        future_data = []
        current_data = self.df[self.df['YEAR'] == (start_year - 1)].copy()

        for year in range(start_year, end_year + 1):
            next_df = self._prepare_next_step(current_data, year)

            X_future = next_df[self.features]
            preds = self._predict(X_future)

            next_df['ENROLLMENT'] = np.maximum(0, preds)

            future_data.append(next_df)
            current_data = next_df.copy()
            print(f" > Forecasted {year}")

        return pd.concat(future_data, ignore_index=True)

    def _prepare_next_step(self, prev_df, target_year):
        next_df = prev_df.copy()
        next_df['YEAR'] = target_year

        # Shift Lags
        next_df['LAG_2'] = next_df['LAG_1']
        next_df['LAG_1'] = next_df['ENROLLMENT']

        # Update Growth
        next_df['YOY_GROWTH'] = (next_df['LAG_1'] - next_df['LAG_2']) / (next_df['LAG_2'] + 1e-5)

        # Update Cohort Logic
        cohort_lookup = prev_df.set_index(['REGION', 'COUNCIL', 'SUBJECT', 'FORM_NUM'])['ENROLLMENT'].to_dict()

        def get_cohort(row):
            target_form = row['FORM_NUM'] - 1
            if target_form < 1: return -1
            key = (row['REGION'], row['COUNCIL'], row['SUBJECT'], target_form)
            return cohort_lookup.get(key, -1)

        next_df['COHORT_LAG'] = next_df.apply(get_cohort, axis=1)
        next_df['IS_ELECTION_YEAR'] = 1 if target_year in [2025, 2030] else 0

        return next_df


# =============================================================================
# MAIN PIPELINE
# =============================================================================

def main():
    # 1. Configuration
    BASE_DIR = '/content/drive/MyDrive/GUIDELINES_TSC_JAN2026/Data Set/csvs/'
    GEO_FILE = '/content/drive/MyDrive/MOEST/tanzania_council_geodata.csv'
    EXCLUDE_KEYWORDS = ['Primary', 'Textbooks', 'Population', 'Teacher', 'COBET', 'Vocational']

    # 2. Load Data
    loader = MOESTDataLoader(BASE_DIR, EXCLUDE_KEYWORDS)
    loader.mount_drive()
    loader.load_data()
    all_dfs = loader.get_all_dataframes()

    # 3. Location & Clean Up
    loc_manager = LocationManager(all_dfs, GEO_FILE)
    loc_manager.standardize_location_columns()
    loc_manager.merge_lga_status()

    # 4. Feature Engineering
    print("\n--- Starting Feature Engineering ---")
    df_subject = all_dfs.get("Secondary_students_per_subject")
    df_tables = all_dfs.get("Data-Secondary Tables and chairs 2016-2025")
    df_labs = all_dfs.get("Combined_Secondary_Laboratories_All_G_NG")

    if df_subject is None:
        raise ValueError("Critical DataFrame 'Secondary_students_per_subject' not found.")

    engineer = FeatureEngineer()
    long_df = engineer.melt_subjects(df_subject)
    merged_df = engineer.merge_infrastructure(long_df, {'tables': df_tables, 'labs': df_labs})
    lagged_df = engineer.create_lag_features(merged_df)
    final_df = engineer.create_cohort_features(lagged_df)

    # 5. Modeling & Forecasting (Updated Strategy)
    print("\n--- Starting Model Engine ---")
    engine = EnrollmentModelEngine(final_df)
    engine.preprocess()

    # Train (Using data up to 2023 to test on 2024 and 2025)
    engine.train_all_models(cutoff_year=2023)

    # Evaluate & Select Strategy (Best or Ensemble)
    engine.evaluate_and_select_strategy(test_year_start=2024)

    # 6. Generate Forecast
    # Note: Recursive forecast starts from 2026, using 2025 as the base history
    forecast_df = engine.recursive_forecast(2026, 2030)

    # 7. Output
    print("\n--- Final Forecast Sample ---")
    output_cols = ['YEAR', 'REGION', 'COUNCIL', 'SUBJECT', 'FORM_NUM', 'ENROLLMENT']
    final_view = forecast_df[output_cols].copy()
    final_view['ENROLLMENT'] = final_view['ENROLLMENT'].round(0).astype(int)
    print(final_view.head(10))

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Found 13 files to load.
Loaded: Data-Secondary Enrollment 2016-2025
Loaded: Dropout-Secondary  2017-2024
Loaded: Data-Secondary Tables and chairs 2016-2025
Loaded: Secondary-Re_entry
Loaded: Secondary - DISABALITY 2020-2025
Loaded: LGAs Urban and Rural Status
Loaded: Combined_Secondary_Laboratories_Govt
Loaded: Combined_Secondary_Laboratories_All_G_NG
Loaded: Combined_Secondary_ICT_All_G_NG
Loaded: Combined_Secondary_ICT_Govt
Loaded: Combined_Secondary_Electricity_All_G_NG
Loaded: Combined_Secondary_Electricity_Govt
Loaded: Secondary_students_per_subject
Merged LGA Status into Data-Secondary Enrollment 2016-2025
Merged LGA Status into Dropout-Secondary  2017-2024
Merged LGA Status into Data-Secondary Tables and chairs 2016-2025
Merged LGA Status into Secondary-Re_entry
Merged LGA Status into Secondary - DISABALITY 2020-2025
Merged LGA Status into Combined_S